# Equal-Weight VN30



## Library Imports



In [1]:
import pandas as pd
import numpy as np
import requests
import math
from vnstock3 import *
import xlsxwriter

In [3]:
stock = Vnstock().stock(symbol='ACB', source='VCI')

2024-09-26 16:15:15,341 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [4]:
# from mysecrets import A_Alpha_Vantage

In [5]:
# symbol = 'IBM'
# url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&outputsize=full&apikey={A_Alpha_Vantage}'
# data = requests.get(url).json()
# data


## Tạo Dataframe những công ty trong VN30 với ba trường: Ticker, Price & MarketCap

### Tạo dataframe cho list công ty trong VN30

In [6]:
stocks = stock.listing.symbols_by_group('VN30')
stock_df = pd.DataFrame(stocks)
stock_df.rename(columns = {'symbol': 'Ticker'}, inplace = True)
print(stock_df)

   Ticker
0     ACB
1     BCM
2     BID
3     BVH
4     CTG
5     FPT
6     GAS
7     GVR
8     HDB
9     HPG
10    MBB
11    MSN
12    MWG
13    PLX
14    POW
15    SAB
16    SHB
17    SSB
18    SSI
19    STB
20    TCB
21    TPB
22    VCB
23    VHM
24    VIB
25    VIC
26    VJC
27    VNM
28    VPB
29    VRE


### In ra gía của cổ phiếu, ví dụ: ACB

In [7]:
ticker = "ACB"
price = float(stock.quote.intraday(symbol = f'{ticker}', show_log = False).iloc[0,1]) / 1000
# price = np.array([price])
print(price)

26.15


### In ra Market Cap, đơn vị: tỷ đồng

In [8]:
x = float(stock.trading.price_board(['ACB']).iloc[0,7])
x1= x/1000000000
x1
marketCap = round(price * x1,2)
marketCap

116.8

### Tạo Dataframe mới với ba trường: Ticker, Price & MarketCap

In [9]:
my_columns = ['Ticker', 'Price', 'MarketCap', 'Number of Shares to Buy']
final_df = pd.DataFrame(columns = my_columns)
final_df

,Ticker,Price,MarketCap,Number of Shares to Buy


### Thêm mã cổ phiếu, giá, vốn hóa vào dataframe mới

In [10]:
new_row = pd.Series([ticker, price, marketCap, 'N/A'], index=my_columns)
final_df = pd.concat([final_df, pd.DataFrame([new_row])], ignore_index=True)
final_df

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


,Ticker,Price,MarketCap,Number of Shares to Buy
0,ACB,26.15,116.8,N/A


### RESULTS

In [11]:
my_columns = ['Ticker', 'Price', 'MarketCap', 'Number of Shares to Buy']
final_df = pd.DataFrame(columns=my_columns)

# lẶP QUA TỪNG TICKER TRONG STOCK_DF
for ticker in stock_df['Ticker']:
    try:
        # LẤY PRICE CHO TỪNG CỔ PHIẾU
        price = float(stock.quote.intraday(symbol=f'{ticker}', show_log=False).iloc[0, 1])
        
        # LẤY MARKETCAP CHO TỪNG CỔ PHIẾU
        x = float(stock.trading.price_board([ticker]).iloc[0, 7])
        x1 = x /1000000000# Adjust for billions
        marketCap = round(price * x1, 2)
        
        # TẠO MỘT HÀNG MỚI VỚI TICKER, PRICE, MARKETCAP
        new_row = pd.Series([ticker, price, marketCap, 'N/A'], index=my_columns)
        
        # THÊM ROW VÀO FINAL_DF
        final_df = pd.concat([final_df, pd.DataFrame([new_row])], ignore_index=True)
    
    except Exception as e:
        print(f"Error with ticker {ticker}: {e}")

final_df


The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


,Ticker,Price,MarketCap,Number of Shares to Buy
0,ACB,26150.0,116803.10,N/A
1,BCM,72000.0,74520.00,N/A
2,BID,50100.0,285591.84,N/A
3,BVH,43400.0,32216.50,N/A
4,CTG,36350.0,195199.20,N/A
5,FPT,134200.0,195992.13,N/A
6,GAS,74400.0,174294.99,N/A
7,GVR,36100.0,144399.24,N/A
8,HDB,28100.0,81704.46,N/A
9,HPG,26150.0,167261.94,N/A


## Tính số lượng cổ phiếu cần mua


In [12]:
portfolio_size = input("Nhập giá trị danh mục:")

try:
    val = float(portfolio_size)
except ValueError:
    print("Invalid \n Try again:")
    portfolio_size = input("Nhập giá trị danh mục:")

In [13]:
position_size = float(portfolio_size) / len(final_df.index)
for i in range(0, len(final_df['Ticker'])):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_df['Price'][i])
final_df

,Ticker,Price,MarketCap,Number of Shares to Buy
0,ACB,26150.0,116803.10,12
1,BCM,72000.0,74520.00,4
2,BID,50100.0,285591.84,6
3,BVH,43400.0,32216.50,7
4,CTG,36350.0,195199.20,9
5,FPT,134200.0,195992.13,2
6,GAS,74400.0,174294.99,4
7,GVR,36100.0,144399.24,9
8,HDB,28100.0,81704.46,11
9,HPG,26150.0,167261.94,12
